In [2]:
import altair as alt
import numpy as np
import pandas as pd

from datetime import timedelta

In [3]:
chosen_owners = ["Eli", "Heather", "Kensie", "Sam", "Rachel"]
#chosen_owners = ["Eli", "Sam", "Rachel"]
days = 140

In [4]:
# Load Data
df = pd.read_json("workitems.json")
df["iterationStartDate"] = pd.to_datetime(df["iterationStartDate"])
df["workStartDate"] = pd.to_datetime(df["workStartDate"])
df["finishDate"] = pd.to_datetime(df["finishDate"])

In [5]:
df

,name,points,owner,iterationStartDate,workStartDate,finishDate
0,Calibrate the Kubernetes Cluster,2,Heather,2019-07-08,2019-07-10,2019-07-10
1,Reimplement the Cache,5,Sam,2019-07-08,2019-07-09,2019-07-16
2,Restore the Secret Store,5,Heather,2019-07-08,2019-07-08,2019-07-10
3,Calibrate the Feature Flag for B,3,Heather,2019-07-08,2019-07-08,2019-07-08
4,Address the Mongo Backend,21,Eli,2019-07-08,2019-07-08,2019-07-31
...,...,...,...,...,...,...
114,Find the Feature Flag for B,3,Kensie,2019-11-25,2019-11-26,2019-11-27
115,Address the CLI Frontend,3,Kensie,2019-11-25,2019-11-25,2019-11-26
116,Address the CLI Frontend,13,Eli,2019-11-25,2019-11-25,2019-11-27
117,Retest the Neo4j Backend,2,Heather,2019-11-25,2019-11-25,2019-11-26


In [6]:
# Filter Data based on `chosen_owners` and `days`
filtered = df[df.owner.isin(chosen_owners)]

end_date = filtered["iterationStartDate"].max()
start_date = end_date - timedelta(days)

filtered = filtered[filtered.iterationStartDate.between(start_date, end_date)]

In [ ]:
#df

In [7]:
# Wrangle data to get cumulative points over time
date_range = pd.date_range(start_date, end_date)
points_over_time = pd.pivot_table(filtered, values="points", index=["finishDate"], columns=["owner"], aggfunc=np.sum, fill_value=0)
points_over_time = points_over_time.reindex(date_range, fill_value=0)
points_over_time = points_over_time.cumsum()
display(points_over_time)
points_over_time = points_over_time.T.reset_index()
points_over_time = pd.melt(points_over_time, id_vars=["owner"], var_name="date", value_name="points")

owner,Eli,Heather,Kensie,Rachel,Sam
2019-07-08,0,3,0,0,0
2019-07-09,0,3,0,0,2
2019-07-10,0,10,0,0,2
2019-07-11,0,10,0,0,2
2019-07-12,0,10,0,5,2
...,...,...,...,...,...
2019-11-21,204,178,176,164,96
2019-11-22,204,178,189,164,96
2019-11-23,204,178,189,164,96
2019-11-24,204,178,189,164,96


In [8]:
points_over_time

,owner,date,points
0,Eli,2019-07-08,0
1,Heather,2019-07-08,3
2,Kensie,2019-07-08,0
3,Rachel,2019-07-08,0
4,Sam,2019-07-08,0
...,...,...,...
700,Eli,2019-11-25,225
701,Heather,2019-11-25,191
702,Kensie,2019-11-25,189
703,Rachel,2019-11-25,164


# Eli Newguy Performance Review

Judging by story points, I went from worst to first in 4 months!

In [16]:
alt.Chart(points_over_time).mark_bar().encode(
    x = 'date',
    y = 'points',
    color = 'owner'
)

alt.Chart(...)

## Interactivity in Jupyter

Requires [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/index.html) to be installed into your Python environment and Jupyter instance

In [13]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

### Interactive Filtering of Data

In [14]:
@interact
def filter_df(owner=["Eli", "Heather", "Kensie", "Rachel", "Sam"], days=(14, 150, 7)):
    chosen_owners = [owner]
    filtered = df[df.owner.isin(chosen_owners)]

    end_date = filtered["iterationStartDate"].max()
    start_date = end_date - timedelta(days)
    return filtered[filtered.iterationStartDate.between(start_date, end_date)]

interactive(children=(Dropdown(description='owner', options=('Eli', 'Heather', 'Kensie', 'Rachel', 'Sam'), val…

### Interactive plot of story points over time

In [15]:
@interact
def filter_df(owner=["Eli", "Heather", "Kensie", "Rachel", "Sam"], days=(14, 150, 7)):
    chosen_owners = [owner]
    filtered = df[df.owner.isin(chosen_owners)]

    end_date = filtered["iterationStartDate"].max()
    start_date = end_date - timedelta(days)
    filtered = filtered[filtered.iterationStartDate.between(start_date, end_date)]
    
    # Wrangle data to get cumulative points over time
    date_range = pd.date_range(start_date, end_date)
    points_over_time = pd.pivot_table(filtered, values="points", index=["finishDate"], columns=["owner"], aggfunc=np.sum, fill_value=0)
    points_over_time = points_over_time.reindex(date_range, fill_value=0)
    points_over_time = points_over_time.cumsum()
    points_over_time = points_over_time.T.reset_index()
    points_over_time = pd.melt(points_over_time, id_vars=["owner"], var_name="date", value_name="points")
    
    return alt.Chart(points_over_time).mark_line().encode(
        x = 'date',
        y = 'points',
        color = 'owner'
    )


interactive(children=(Dropdown(description='owner', options=('Eli', 'Heather', 'Kensie', 'Rachel', 'Sam'), val…